In [14]:
import pandas as pd
import gensim 
from gensim.models import Word2Vec 
from tqdm import tqdm
import re
import numpy as np
  
dataset = pd.read_csv("data/names_combined.csv")

dataset.dropna(inplace=True)
dataset.replace('b','black',inplace=True)
  
data = [] 
corpus = []

def features(name):
    bigrams = []
    trigrams = []
    for i in range(len(name)-1):
        bigrams.append(name[i:i+2])
        if i < len(name) - 2:
            trigrams.append(name[i:i+3])
    trigrams.append(name[-3:])
    fets = bigrams+trigrams
    return fets

for i in tqdm(range(len(dataset['name']))):
    temp = []
    review = re.sub('[^a-zA-Z]', ' ', dataset['name'].values[i])
    review = re.sub('  ', ' ', review)
    review = review.lower()
    if review[0] == ' ':
        review=review[1:]
    temp = list(review) + features(review)
    data.append(temp)
    corpus.append(review)

100%|██████████| 120719/120719 [00:02<00:00, 45515.73it/s]


In [15]:
# Create & train Char vec model
model1 = gensim.models.Word2Vec(data, min_count = 1, size = 300, window = 10) ## 300,10
print('model created')
model1.train(data,total_examples=model1.corpus_count, epochs=15)
print('model trained')

model created
model trained


In [16]:
y = dataset.iloc[:, 2].values
X = []
for c in tqdm(corpus):
	vecs = model1[list(c) + features(c)]
	X.append(vecs.sum(axis=0))
X = np.array(X)

  0%|          | 0/120719 [00:00<?, ?it/s]/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.
100%|██████████| 120719/120719 [00:22<00:00, 5293.39it/s]


In [17]:
from imblearn.over_sampling import SMOTE, RandomOverSampler
ros = RandomOverSampler(random_state=42)
X, y = ros.fit_resample(X, y)

In [18]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
y = enc.fit_transform(y.reshape(-1,1)).toarray()

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)#, random_state = 0)

In [20]:
#Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [21]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, GRU, LSTM, Flatten
from keras.layers.normalization import BatchNormalization
from keras import optimizers

classifier = Sequential()

######## race classification 87% test accuracy
classifier.add(Dense(input_dim=300, output_dim=256, activation='relu'))
classifier.add(BatchNormalization())
classifier.add(Dropout(0.2))  # Dropout overfitting
classifier.add(Dense(output_dim=256, activation='relu'))
classifier.add(BatchNormalization())
classifier.add(Dropout(0.1))
classifier.add(Dense(output_dim=128, activation='relu'))
classifier.add(BatchNormalization())
classifier.add(Dropout(0.1))
classifier.add(Dense(output_dim=128, activation='relu'))
classifier.add(BatchNormalization())
classifier.add(Dropout(0.1))
classifier.add(Dense(output_dim=64, activation='relu'))
classifier.add(BatchNormalization())
classifier.add(Dropout(0.1))
classifier.add(Dense(output_dim=32, activation='relu'))
classifier.add(BatchNormalization())
classifier.add(Dropout(0.1))
classifier.add(Dense(output_dim=4, activation='softmax'))

##### gender classification model 98% test acc
# classifier1.add(Dense(input_dim=300, output_dim=256, activation='relu'))
# classifier1.add(BatchNormalization())
# classifier1.add(Dropout(0.1))  # Dropout overfitting
# classifier1.add(Dense(output_dim=128, activation='relu'))
# classifier1.add(BatchNormalization())
# classifier1.add(Dropout(0.1))
# classifier1.add(Dense(output_dim=128, activation='relu'))
# classifier1.add(BatchNormalization())
# classifier1.add(Dropout(0.1))
# classifier1.add(Dense(output_dim=64, activation='relu'))
# classifier1.add(BatchNormalization())
# classifier1.add(Dropout(0.1))
# classifier1.add(Dense(output_dim=64, activation='relu'))
# classifier1.add(BatchNormalization())
# classifier1.add(Dropout(0.1))
# classifier1.add(Dense(output_dim=32, activation='relu'))
# classifier1.add(BatchNormalization())
# classifier1.add(Dropout(0.1))
# classifier1.add(Dense(output_dim=2, activation='softmax'))


classifier.compile(loss="categorical_crossentropy", optimizer="adam", metrics = ['accuracy'])
history = classifier.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=40, batch_size=256)

# classifier1.compile(loss="categorical_crossentropy", optimizer="adam", metrics = ['accuracy'])
# history1 = classifier1.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=60, batch_size=256)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=300, activation="relu", units=256)`
  # Remove the CWD from sys.path while we load stuff.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:13: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=256)`
  del sys.path[0]
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:16: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=128)`
  app.launch_new_instance()
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:19: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=128)`
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykern

Train on 155664 samples, validate on 38916 samples
Epoch 1/40
155664/155664 [==============================] - 15s 97us/step - loss: 0.5802 - acc: 0.7486 - val_loss: 0.4507 - val_acc: 0.8050
Epoch 2/40
155664/155664 [==============================] - 11s 68us/step - loss: 0.4620 - acc: 0.8020 - val_loss: 0.4057 - val_acc: 0.8262
Epoch 3/40
155664/155664 [==============================] - 11s 72us/step - loss: 0.4253 - acc: 0.8180 - val_loss: 0.3845 - val_acc: 0.8365
Epoch 4/40
155664/155664 [==============================] - 11s 68us/step - loss: 0.4028 - acc: 0.8276 - val_loss: 0.3775 - val_acc: 0.8395
Epoch 5/40
155664/155664 [==============================] - 11s 68us/step - loss: 0.3912 - acc: 0.8339 - val_loss: 0.3648 - val_acc: 0.8453
Epoch 6/40
155664/155664 [==============================] - 10s 67us/step - loss: 0.3725 - acc: 0.8403 - val_loss: 0.3529 - val_acc: 0.8501
Epoch 7/40
155664/155664 [==============================] - 10s 66us/step - loss: 0.3620 - acc: 0.8455 - val_

In [22]:
def predict_gen(name):
    name = name.lower()
    classes = ['female','male']
    namevec = model1[list(name)+features(name)].sum(axis=0).flatten()
    pred = classifier1.predict(namevec.reshape(1,300))
    predprob = pred[0][np.argmax(pred)]*100
    gender = classes[np.argmax(pred)]
    return f'{name} is probably a {gender}.... chances:{predprob}%'

def predict_race(name):
    try:
        name = name.lower()
        classes = ['black','hispanic','indian','white']
        namevec = model1[list(name)+features(name)].sum(axis=0).flatten()
        pred = classifier.predict(sc.transform(namevec.reshape(1,300)))
        predprob = pred[0][np.argmax(pred)]*100
        race = classes[np.argmax(pred)]
    except:
        return "None"
    return f'{name} is {race}....{predprob}%.....probs:{pred[0]} for {classes} respectively'

In [37]:
predict_race('juliya')

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


"juliya is indian....99.9987006187439%.....probs:[1.3417108e-06 2.5491079e-06 9.9998701e-01 9.1079673e-06] for ['black', 'hispanic', 'indian', 'white'] respectively"

In [24]:
y_pred = classifier.predict(X_test)

In [25]:
classes = ['black','hispanic','indian','white']
test_class = []
pred_class = []
for i in range(len(y_pred)):
    test_class.append([classes[np.argmax(y_test[i])]])
    pred_class.append([classes[np.argmax(y_pred[i])]])

In [26]:
from sklearn.metrics import confusion_matrix, classification_report
cm = confusion_matrix(test_class, pred_class)
print(cm)
print(classification_report(test_class, pred_class,target_names=classes))

[[7783   82   16 1880]
 [   0 9780    0    6]
 [  10    2 9753   18]
 [1856  844   36 6850]]
              precision    recall  f1-score   support

       black       0.81      0.80      0.80      9761
    hispanic       0.91      1.00      0.95      9786
      indian       0.99      1.00      1.00      9783
       white       0.78      0.71      0.75      9586

   micro avg       0.88      0.88      0.88     38916
   macro avg       0.87      0.88      0.87     38916
weighted avg       0.87      0.88      0.88     38916



In [ ]:
# import keras
# from keras.models import Sequential
# from keras.layers import Dense, Dropout, LSTM, Flatten
# from keras.layers.normalization import BatchNormalization

# model = Sequential()
# model.add(LSTM(300, return_sequences=True, input_shape=(30,28)))
# model.add(Dropout(0.2))
# model.add(LSTM(300, return_sequences=False))
# model.add(Dropout(0.1))
# model.add(Dense(output_dim=32, activation='relu'))
# model.add(BatchNormalization())
# model.add(Dropout(0.1))
# model.add(Dense(2, activation='softmax'))
# model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

# history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=2048)